In [1]:
# Append the src folder to the path to access helper functions in src folder
import sys
path_to_src_folder = "../src"
sys.path.append(path_to_src_folder)

from dotenv import dotenv_values
import requests
from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel
from helpers.writer_citer_model import WriterCiterModel
from helpers.visibility import parse_citations_from_text, compute_citation_scores

"""
============================================
Interact with Helpers to mimic Perplexity
============================================
"""

"""
Set up
"""

# Number of links to consider
K = 5

# Select a query to use for the current trial
query = "Why should I buy life insurance?"

# Load environment variables to instantiate helpers
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
writer_citer = WriterCiterModel(api_key=config['OPEN_AI_API_KEY'])

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=K)

"""
View links
"""

top_k_links = [item['link'] for item in search_results['items']][:K]
top_k_links

['https://www.newyorklife.com/articles/six-reasons-to-buy-life-insurance',
 'https://www.reddit.com/r/FinancialPlanning/comments/12vb4s6/do_any_of_you_have_life_insurance_and_is_there/',
 'https://www.iii.org/article/why-should-i-buy-life-insurance',
 'https://www.reddit.com/r/Insurance/comments/17n2csg/how_many_people_have_life_insurance_and_is_it/',
 'https://idoi.illinois.gov/consumers/consumerinsurance/lifeannuities/buying-life-insurance.html']

In [2]:
"""
Fetch the raw text from the html for each link
"""

raw_text_from_html = []

# For each item of raw text that was stored ...
for link in top_k_links:    

    try:

        # Fetch the HTML associated with the link
        response = requests.get(link)
    
        # Get the raw text associated with the HTML
        parsed_text = HTMLParser.parse_html(response.content)
    
        # Store the raw text from the HTML
        raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})

    except:
        print("Something happened with ", link)


"""
Get the refined text from the raw text
"""


website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")


"""
Print the refined text
"""
for item in website_refiner_responses:
    print(item['link'])
    print(item['choices'][0]['message']['content'])
    print()


https://www.newyorklife.com/articles/six-reasons-to-buy-life-insurance
Reasons to buy life insurance are different for everyone. But the decision to purchase insurance is, at its core, all about providing financial security for yourself and the ones you care about. Here are some points on why life insurance is important:

1. **Financial Security**: Life insurance protects your spouse and children from potentially devastating financial losses that could result from your death. It provides financial security to cover debts, living expenses, and medical or final expenses.

2. **Guaranteed Protection**: Life insurance benefits act as a financial safety net for your dependents, ensuring they receive a lump-sum payment when you die, which provides essential protection.

3. **Income Replacement**: If your income were to suddenly disappear, life insurance can help ensure your loved ones have the money they need to cover mortgage, childcare, health care, tuition, and debt.

4. **Tax-Free Benefi

In [3]:
"""
Bundle up a set of reference material to send to the writer-citer model to answer query
"""

reference_material =  [
    {
        "link" : item['link'],
        "information" : item['choices'][0]['message']['content']
    }
    for item in website_refiner_responses
]

reference_material

[{'link': 'https://www.newyorklife.com/articles/six-reasons-to-buy-life-insurance',
  'information': 'Reasons to buy life insurance are different for everyone. But the decision to purchase insurance is, at its core, all about providing financial security for yourself and the ones you care about. Here are some points on why life insurance is important:\n\n1. **Financial Security**: Life insurance protects your spouse and children from potentially devastating financial losses that could result from your death. It provides financial security to cover debts, living expenses, and medical or final expenses.\n\n2. **Guaranteed Protection**: Life insurance benefits act as a financial safety net for your dependents, ensuring they receive a lump-sum payment when you die, which provides essential protection.\n\n3. **Income Replacement**: If your income were to suddenly disappear, life insurance can help ensure your loved ones have the money they need to cover mortgage, childcare, health care, tui

In [4]:
# Generate a cited response with the writer-citer model
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=reference_material
)


In [5]:
text = cited_response['choices'][0]['message']['content']
print(text)

Life insurance offers multiple benefits that make it a crucial component of sound financial planning. Firstly, it provides financial security to your dependents by protecting them from potentially devastating financial losses upon your death, covering debts, living expenses, and final expenses like medical bills and funeral costs [1]. This coverage ensures your family can maintain their standard of living even in your absence [5].

Additionally, life insurance serves as a replacement for lost income, helping ensure that your loved ones have the necessary funds to cover expenses such as mortgage payments, childcare, healthcare, and tuition [1][3]. Beyond merely replacing income, life insurance can also facilitate inheritance, even if there are no other assets to pass on, providing an inheritance to beneficiaries [3].

Moreover, life insurance can be a tax-efficient way to provide for your heirs, as benefits are typically passed on federal income tax-free, which can also help pay any app

In [6]:
# Compute the visibility scores for the reference material
parsed_data = parse_citations_from_text(text)
scores = compute_citation_scores(parsed_data, num_citations=5, normalize=False)
normalized_scores = compute_citation_scores(parsed_data, num_citations=5, normalize=True)

In [7]:
scores

[65.13928145132705, 0, 49.48624632523282, 0, 10.409586237918505]

In [8]:
normalized_scores

[0.5209679054140286, 0.0, 0.3957787915441304, 0.0, 0.08325330304184096]